# 	1.	🔧 !pip install torch joblib

In [ ]:
!pip install torch joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 803.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

# 2. 	📊 Train model


In [ ]:
# train_mlp_classifier.py

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# 1. Load or generate sample data
# Format: [car_length, car_width, spot_length, spot_width, distance_to_spot], label

data = pd.DataFrame([
    [4.7, 1.8, 5.5, 2.5, 1.0, 1],  # Toyota Corolla
    [4.9, 2.0, 6.0, 2.5, 0.8, 0],  # Honda Accord
    [5.2, 2.0, 5.0, 2.0, 0.3, 2],  # Ford Explorer – too small
    [3.9, 1.6, 4.8, 2.2, 1.2, 1],  # Mini Cooper
    [4.3, 1.7, 5.0, 2.3, 0.7, 0],  # Hyundai i20
    [5.0, 2.2, 6.5, 2.6, 1.0, 0],  # Tesla Model S
    [4.6, 1.9, 5.2, 2.4, 0.9, 1],  # Mazda 3 hatch
    [5.1, 2.1, 4.9, 2.0, 0.2, 2],  # Jeep Grand Cherokee
    [4.4, 1.8, 5.5, 2.5, 1.1, 1],  # Nissan Sentra
    [4.8, 2.0, 6.0, 2.5, 1.3, 0],  # VW Passat
    [4.5, 1.8, 6.0, 2.5, 1.0, 1],
    [4.8, 2.0, 5.5, 2.3, 0.5, 0],
    [5.0, 2.2, 4.5, 2.0, 0.3, 2],
    [4.3, 1.7, 6.0, 2.4, 1.2, 1],
    [5.2, 2.1, 5.0, 2.0, 0.4, 0],
    [5.5, 2.3, 4.8, 2.0, 0.2, 2]
], columns=["car_length", "car_width", "spot_length", "spot_width", "distance_to_spot", "label"])

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# 2. Normalize input
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler for later use
# ✅ Create 'models/' folder first
import os
os.makedirs("models", exist_ok=True)

# 💾 Save scaler
import joblib
joblib.dump(scaler, "models/scaler.pkl")

# 3. Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# 4. Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

# 5. Define model
model = nn.Sequential(
    nn.Linear(5, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 3)
)

# 6. Train model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🏋️‍♀️ Train the model
print("🚀 Training started...")
for epoch in range(100):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

print("✅ Training completed.")

# 7. Save model
torch.save(model.state_dict(), "models/mlp_strategy.pt")
print("✅ Model saved to models/mlp_strategy.pt")

🚀 Training started...
Epoch 0, Loss: 1.0731
Epoch 10, Loss: 1.0223
Epoch 20, Loss: 0.9694
Epoch 30, Loss: 0.9096
Epoch 40, Loss: 0.8449
Epoch 50, Loss: 0.7792
Epoch 60, Loss: 0.7145
Epoch 70, Loss: 0.6540
Epoch 80, Loss: 0.5988
Epoch 90, Loss: 0.5477
✅ Training completed.
✅ Model saved to models/mlp_strategy.pt


📉 What Is “Loss”?

Loss is a number that tells you how far off your model’s predictions are from the correct answers during training.

	•	✅ Lower loss = better model performance
	•	❌ Higher loss = worse predictions

You want the loss to go down as the model learns.

- Epoch 0, Loss: 1.1076   ← ❌ High loss, model just started
- Epoch 10, Loss: 1.0091  ← starting to improve
- Epoch 20, Loss: 0.9300
- Epoch 30, Loss: 0.8503
- Epoch 40, Loss: 0.7623
- Epoch 50, Loss: 0.6655   ← ✅ Big improvement
- Epoch 60, Loss: 0.5683
- Epoch 70, Loss: 0.4785
- Epoch 80, Loss: 0.4023
- Epoch 90, Loss: 0.3413   ← 🔥 Low loss, model learned well


🧠 This shows that:

	•	Your model learned from the data step-by-step
	•	Loss decreased from 1.1 → 0.34 (very good!)
	•	This means your model is making fewer mistakes and giving more accurate predictions by the end

In [ ]:
!ls

models	sample_data


# 	3.	💾 Predict strategy using reusable function

In [17]:
import torch
import numpy as np
import joblib

# Load the model (should be done only once ideally)
model = torch.nn.Sequential(
    torch.nn.Linear(5, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 3)
)
model.load_state_dict(torch.load('models/mlp_strategy.pt'))
model.eval()

# Try to load scaler
try:
    scaler = joblib.load('models/scaler.pkl')
except:
    scaler = None
    print("⚠️ No scaler used")

# ✅ Clean, reusable function
def predict_strategy(car_length, car_width, spot_length, spot_width, distance, verbose=False):
    """
    Predict parking strategy based on car and spot dimensions.

    Returns:
        - recommendation (str): strategy name
        - confidence (float): softmax probability
        - all_probs (dict): confidence scores for all 3 classes
    """
    sample_input = np.array([[car_length, car_width, spot_length, spot_width, distance]])

    if scaler:
        sample_input = scaler.transform(sample_input)

    input_tensor = torch.tensor(sample_input, dtype=torch.float32)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1).numpy()[0]
        predicted_class = torch.argmax(output, dim=1).item()

    strategy_labels = ['Reverse Parallel', 'Forward Perpendicular', 'Cannot Park']
    recommendation = strategy_labels[predicted_class]
    confidence = probabilities[predicted_class]
    all_probs = {label: float(f"{prob:.4f}") for label, prob in zip(strategy_labels, probabilities)}

    # Optional console print for debugging
    if verbose:
        print("📊 Strategy Confidence Scores:")
        for label, prob in all_probs.items():
            print(f"{label}: {prob*100:.2f}%")
        print(f"\n🚘 Recommended Strategy: {recommendation} ({confidence:.2%} confidence)")

    return recommendation, confidence, all_probs

🔧 How to Use It

In [ ]:
# strategy, confidence, scores = predict_strategy(3.0, 1.6, 5.0, 2.5, 1.0, verbose=True)

# # In Streamlit later:
# st.write("Recommended:", strategy)
# st.write("Confidence:", f"{confidence:.2%}")
# st.json(scores)

In [18]:
strategy, confidence, scores = predict_strategy(3.0, 1.6, 5.0, 2.5, 1.0, verbose=True)

📊 Strategy Confidence Scores:
Reverse Parallel: 3.75%
Forward Perpendicular: 96.02%
Cannot Park: 0.24%

🚘 Recommended Strategy: Forward Perpendicular (96.02% confidence)


In [19]:
strategy, confidence, scores = predict_strategy(4.2, 1.8, 6.5, 2.8, 1.5, verbose=True)

📊 Strategy Confidence Scores:
Reverse Parallel: 17.04%
Forward Perpendicular: 82.31%
Cannot Park: 0.66%

🚘 Recommended Strategy: Forward Perpendicular (82.31% confidence)


In [20]:
# Try a tighter spot
strategy, confidence, scores = predict_strategy(5.0, 2.2, 6.0, 2.8, 0.9, verbose=True)

📊 Strategy Confidence Scores:
Reverse Parallel: 85.31%
Forward Perpendicular: 8.48%
Cannot Park: 6.21%

🚘 Recommended Strategy: Reverse Parallel (85.31% confidence)


In [21]:
# Very small car
strategy, confidence, scores = predict_strategy(3.0, 1.6, 5.0, 2.5, 1.0, verbose=True)

📊 Strategy Confidence Scores:
Reverse Parallel: 3.75%
Forward Perpendicular: 96.02%
Cannot Park: 0.24%

🚘 Recommended Strategy: Forward Perpendicular (96.02% confidence)


In [22]:
strategy, confidence, scores = predict_strategy(4.5, 2.0, 5.5, 2.5, 1.2, verbose=True)

📊 Strategy Confidence Scores:
Reverse Parallel: 41.49%
Forward Perpendicular: 51.26%
Cannot Park: 7.24%

🚘 Recommended Strategy: Forward Perpendicular (51.26% confidence)
